In [2]:
from Crypto.Util.number import *
import itertools

import itertools

def small_roots(f, bounds, m=1, d=None):
	if not d:
		d = f.degree()

	if isinstance(f, Polynomial):
		x, = polygens(f.base_ring(), f.variable_name(), 1)
		f = f(x)

	R = f.base_ring()
	N = R.cardinality()
	
	f /= f.coefficients().pop(0)
	f = f.change_ring(ZZ)

	G = Sequence([], f.parent())
	for i in range(m+1):
		base = N^(m-i) * f^i
		for shifts in itertools.product(range(d), repeat=f.nvariables()):
			g = base * prod(map(power, f.variables(), shifts))
			G.append(g)

	B, monomials = G.coefficients_monomials()
	monomials = vector(monomials)

	factors = [monomial(*bounds) for monomial in monomials]
	for i, factor in enumerate(factors):
		B.rescale_col(i, factor)

	B = B.dense_matrix().LLL()

	B = B.change_ring(QQ)
	for i, factor in enumerate(factors):
		B.rescale_col(i, 1/factor)

	H = Sequence([], f.parent().change_ring(QQ))
	for h in filter(None, B*monomials):
		H.append(h)
		I = H.ideal()
		if I.dimension() == -1:
			H.pop()
		elif I.dimension() == 0:
			roots = []
			for root in I.variety(ring=ZZ):
				root = tuple(R(root[var]) for var in f.variables())
				roots.append(root)
			return roots

	return []


########################################################### data
n = 136964268696705411012931678444400476965128010872555448400213620622067883854478236421918692996629963139047415072472047678976544812525410183376508281777753764772717556749040353369224632837988481501635782332890362346072540376844673583200793306150277865329288352094467307041851680604797780622565902104592357680593
c1 = 107944610477014438368669730240290187641956135255874565912680801847773594167609552986103389387959841325598268708151075742451847739687114032207870408273148403515527380178079660615211473860926874410913965007062103999478602653442039162450485587002132252791726932886248702887481193005508833047658670524174403487926
c2 = 3127498828362590440687643816538507936881211309947428328430490606826730097403278709986417034273659633325080685701236559426358207715864424852124315384588557663762413703102094708413550506326468486129474303727525648090980240869036451589898292059083448235596148583344000305316200155116533939780870821880308265351
c3 = 34750376493440567438722422230088819245126408055599726268659066362718882920292668856236427500542845906563994346099699160835707954070313392712772915426103654356011333383429223545917281329858790718233462130418227815432974371149743106557223575520754522565880639680621339819606200130161409319937540290256997798363
c4 = 68386722499942490423218634548938698850452201789220111973592566020264164055166337559044317692412740033275416658408328173668330116310709068338409465503187081630678057899318905087174940956562363405162595996348729729582926384404944453856631426707948200531179167461049403247731239166275155168326620039916039609160
c5 = 99091859957185268182627104328381847555153474916462106015384341389704949281024811672220439525424224356889662479138565807801371100664549459535092254296125591164696438439192690741081989584585241901233831028084944219073321712141738384374448620289934362539580673210387595927204938373486338706824525375424713025762


########################################################### part1 univariate coppersmith
PR.<x> = PolynomialRing(Zmod(n))
f = (c2 - x)^3 - c1^7
f = f.monic()
res = f.small_roots(X=2^128,beta=1,epsilon=0.05)
a = int(res[0])


def gcd(g1, g2):
    while g2:
        g1, g2 = g2, g1 % g2
    return g1.monic()

PR.<x> = PolynomialRing(Zmod(n))
f1 = x^3 - c1
f2 = x^7 + a - c2
print(long_to_bytes(int(-gcd(f1,f2)[0])))


########################################################### part2 bivariate coppersmith
PR.<t,m,a,b> = PolynomialRing(Zmod(n))
f1 = t - c3
f2 = t^2*m + a - c4
f3 = t^3 + b*m^2 - c5*m^2

g1 = f1.sylvester_matrix(f2,t).det()
g2 = f1.sylvester_matrix(f3,t).det()
h = g1.sylvester_matrix(g2,m).det()
print(h)
PR.<a,b> = PolynomialRing(Zmod(n))
F = a^2*b + 37872408739520142830304574116018629409974535956093342384829279232362934573453424749698253471205738782157752593333481871175173711860860723841416027481628173608021118309847662628142643253403239600401951304805418126999218664702935198826344685860343502789707678884079711114646742231311441915741376729167644654831*a^2 + 190823696820430166494409346523079264223607294115224453028488581539555744145561303830057611804483072496581755655391331639884579903992046699689350771379601511361440950402543194874750924863754691310590340192902886906687608034784675487530452734381464266930017172368500546389202272247470285912662024760278462273*a*b + 80582112561404657449509150081384899909557311869412456743572902879183523289264580866172631537170101896408099312302884886562771169692457383711779147986302460508267407868662435015610971412363243757741897565382241392769325702923510996082165605228192490362483504703747845864818582754068118818114955344107888377314*a + 68093691741396922214190636266815235737657401172706388961290611410127435641706980462045225990095304017113936749903881463677053428347233124981247051813391964827768662209634185526280428515987474898796576950810793328618830835589463378403578874291191571807863823902734723887741412937421463857287610206737239199073*b + 119377616806287408733463729955763808864220271449542182204885972590227167693309302171677114408496837527167197624456182080289687837718246588982805386901201954422315153741576142449690953948702359486189509589120120277429515192826257617131416838168386619391214901432936362526683189061999125023105912877974836659728
res = small_roots(F, (2^128,2^128), m=4,d=4)
a, b = int(res[0][0]), int(res[0][1])



def gcd(g1, g2):
    while g2:
        g1, g2 = g2, g1 % g2
    return g1.monic()

PR.<x> = PolynomialRing(Zmod(n))
f1 = x^333 - c3
f2 = x^667 + a - c4
print(long_to_bytes(int(-gcd(f1,f2)[0])))


#catctf{Un1vaRiate_c0pp3rSm1th_@nd_Biv4r1aTe_C0Pp3rSm1th_4ND_Fin@lLy_P0lyn0m1Als_GCD!}

b'catctf{Un1vaRiate_c0pp3rSm1th_@nd_Biv4r1aT\x10B\x07\xe7\xe3\x92\x05\x87q\xa8=<\xc4e\xc8\xf7\xe3\xe78.\x17\n\xaa\xa4\xa6)e\xd55\x7f/oS\x19\x06\x04!q\xbc:p\xde\x89\xac-#T\xa0\xc9\xdb\x02\xba~B\x8aAvp\xb5\x89\x1fS\xfa\x7fE-$\xeelN\xe6\xde\xe5\x90\xac\xcb\xb0\x9e\xf5\xddd\xc2)\x1a\x0c\xa9'
a^2*b + 37872408739520142830304574116018629409974535956093342384829279232362934573453424749698253471205738782157752593333481871175173711860860723841416027481628173608021118309847662628142643253403239600401951304805418126999218664702935198826344685860343502789707678884079711114646742231311441915741376729167644654831*a^2 + 190823696820430166494409346523079264223607294115224453028488581539555744145561303830057611804483072496581755655391331639884579903992046699689350771379601511361440950402543194874750924863754691310590340192902886906687608034784675487530452734381464266930017172368500546389202272247470285912662024760278462273*a*b + 805821125614046574495091500813848999095573118694124567435729028791835232892